In [1]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd
from time import time

In [2]:
# load data
 
df=pd.read_csv('Waveform_withoutdupl_norm_v01.csv')  
del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)
df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att13,att14,att15,att16,att17,att18,att19,att20,att21,outlier
0,0.323657,0.793785,0.848661,0.660063,0.622244,0.755102,0.760762,0.671655,0.603723,0.636170,...,0.281899,0.203557,0.286607,0.474383,0.352483,0.358857,0.275626,0.465937,0.220884,1
1,0.423803,0.769774,0.632130,0.572471,0.747495,0.670408,0.715232,0.617958,0.637411,0.457447,...,0.302671,0.375494,0.296429,0.383302,0.545278,0.394286,0.451025,0.470803,0.468541,1
2,0.477504,0.580508,0.554133,0.476538,0.851703,0.509184,0.745861,0.785211,0.508865,0.519149,...,0.341246,0.384387,0.229464,0.350095,0.117819,0.406857,0.149203,0.402676,0.370817,1
3,0.645864,0.570621,0.525029,0.631908,0.682365,0.382653,0.478477,0.608275,0.476950,0.190426,...,0.500495,0.555336,0.463393,0.537951,0.564752,0.485714,0.576310,0.518248,0.597055,1
4,0.219158,0.690678,0.530850,0.715328,0.687375,0.571429,0.649007,0.562500,0.523050,0.303191,...,0.190900,0.408103,0.322321,0.425996,0.400195,0.701714,0.515945,0.294404,0.463186,1


In [3]:
df.shape

(3443, 22)

In [4]:
# data to values
df = df.values

In [5]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/waveform.json")
gc = GCForest(config)

In [6]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,21]
X = df[:,0:21]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
# number of class
len(np.unique(y_train))

2

In [8]:
             # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-04-26 15:22:05,062][cascade_classifier.fit_transform] X_groups_train.shape=[(2306, 21)],y_train.shape=(2306,),X_groups_test.shape=[(1137, 21)],y_test.shape=(1137,)
[ 2018-04-26 15:22:05,063][cascade_classifier.fit_transform] group_dims=[21]
[ 2018-04-26 15:22:05,065][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 15:22:05,066][cascade_classifier.fit_transform] group_ends=[21]
[ 2018-04-26 15:22:05,067][cascade_classifier.fit_transform] X_train.shape=(2306, 21),X_test.shape=(1137, 21)
[ 2018-04-26 15:22:05,069][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2306, 21), X_cur_test.shape=(1137, 21)
[ 2018-04-26 15:22:05,612][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=96.97%
[ 2018-04-26 15:22:06,679][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.97%
[ 2018-04-26 15:22:07,729][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

In [9]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-26 15:22:15,393][cascade_classifier.transform] X_groups_test.shape=[(1137, 21)]
[ 2018-04-26 15:22:15,396][cascade_classifier.transform] group_dims=[21]
[ 2018-04-26 15:22:15,397][cascade_classifier.transform] X_test.shape=(1137, 21)
[ 2018-04-26 15:22:15,399][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1137, 21)


Test Accuracy of ExtraTrees = 97.185576 %


In [10]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-26 15:22:18,029][cascade_classifier.transform] X_groups_test.shape=[(1137, 21)]
[ 2018-04-26 15:22:18,031][cascade_classifier.transform] group_dims=[21]
[ 2018-04-26 15:22:18,032][cascade_classifier.transform] X_test.shape=(1137, 21)
[ 2018-04-26 15:22:18,034][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1137, 21)


In [11]:
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-26 15:22:20,493][cascade_classifier.fit_transform] X_groups_train.shape=[(2306, 23)],y_train.shape=(2306,),X_groups_test.shape=[(1137, 23)],y_test.shape=(1137,)
[ 2018-04-26 15:22:20,495][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-04-26 15:22:20,497][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 15:22:20,498][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-04-26 15:22:20,499][cascade_classifier.fit_transform] X_train.shape=(2306, 23),X_test.shape=(1137, 23)
[ 2018-04-26 15:22:20,502][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2306, 23), X_cur_test.shape=(1137, 23)
[ 2018-04-26 15:22:21,424][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.40%
[ 2018-04-26 15:22:22,375][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.84%
[ 2018-04-26 15:22:23,467][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

In [12]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-26 15:22:31,340][cascade_classifier.transform] X_groups_test.shape=[(1137, 23)]
[ 2018-04-26 15:22:31,342][cascade_classifier.transform] group_dims=[23]
[ 2018-04-26 15:22:31,344][cascade_classifier.transform] X_test.shape=(1137, 23)
[ 2018-04-26 15:22:31,346][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1137, 23)


Test Accuracy of ExtraTrees = 97.889182 %


In [13]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')

In [14]:
# Deep
for i in range (20):
    t0 = time()
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    tt = time() - t0
    affiche() 
    print (" Time ",format(round(tt,3))," seconds")

[ 2018-04-26 15:22:34,000][cascade_classifier.fit_transform] X_groups_train.shape=[(2306, 23)],y_train.shape=(2306,),X_groups_test.shape=[(1137, 23)],y_test.shape=(1137,)
[ 2018-04-26 15:22:34,002][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-04-26 15:22:34,004][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 15:22:34,005][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-04-26 15:22:34,006][cascade_classifier.fit_transform] X_train.shape=(2306, 23),X_test.shape=(1137, 23)
[ 2018-04-26 15:22:34,008][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2306, 23), X_cur_test.shape=(1137, 23)
[ 2018-04-26 15:22:34,854][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.40%
[ 2018-04-26 15:22:36,065][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.27%
[ 2018-04-26 15:22:37,167][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

('Test Accuracy of ExtraTrees = 98.065084 %', 0, 'fois')
(' Time ', '10.951', ' seconds')


[ 2018-04-26 15:22:48,297][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.40%
[ 2018-04-26 15:22:49,383][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.27%
[ 2018-04-26 15:22:50,464][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.27%
[ 2018-04-26 15:22:51,533][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.27%
[ 2018-04-26 15:22:52,627][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.27%
[ 2018-04-26 15:22:53,715][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.70%
[ 2018-04-26 15:22:54,811][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.84%
[ 2018-04-26 15:22:55,939][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.83%


('Test Accuracy of ExtraTrees = 97.977133 %', 1, 'fois')
(' Time ', '11.002', ' seconds')


[ 2018-04-26 15:23:01,785][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.40%
[ 2018-04-26 15:23:02,731][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.84%
[ 2018-04-26 15:23:03,735][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.27%
[ 2018-04-26 15:23:04,852][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.84%
[ 2018-04-26 15:23:06,041][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.27%
[ 2018-04-26 15:23:06,899][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.70%
[ 2018-04-26 15:23:07,977][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.84%
[ 2018-04-26 15:23:09,026][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.83%


...

('Test Accuracy of ExtraTrees = 97.889182 %', 17, 'fois')
(' Time ', '10.753', ' seconds')


[ 2018-04-26 15:26:32,740][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.40%
[ 2018-04-26 15:26:33,806][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.84%
[ 2018-04-26 15:26:34,886][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.84%
[ 2018-04-26 15:26:35,824][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.40%
[ 2018-04-26 15:26:36,890][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.27%
[ 2018-04-26 15:26:37,957][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.84%
[ 2018-04-26 15:26:39,009][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.84%
[ 2018-04-26 15:26:40,071][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.39%


('Test Accuracy of ExtraTrees = 97.713281 %', 18, 'fois')
(' Time ', '10.429', ' seconds')


[ 2018-04-26 15:26:45,804][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.40%
[ 2018-04-26 15:26:46,951][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.27%
[ 2018-04-26 15:26:48,050][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.84%
[ 2018-04-26 15:26:49,146][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.40%
[ 2018-04-26 15:26:50,313][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.27%
[ 2018-04-26 15:26:51,392][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.84%
[ 2018-04-26 15:26:52,459][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.84%
[ 2018-04-26 15:26:53,732][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.83%


('Test Accuracy of ExtraTrees = 97.625330 %', 19, 'fois')
(' Time ', '11.322', ' seconds')


# ('Test Accuracy of ExtraTrees = 98.065084 %', 0, 'fois')

# (' Time ', '10.951', ' seconds')